# Training best resnet based on hyperopt results

In [1]:
from urllib.request import urlopen
from fastai.vision import *
from fastai.metrics import error_rate
from PIL import Image
#from google.colab import drive
from os import listdir
import numpy as np 
import cv2
from matplotlib import image
from matplotlib import pyplot
import numpy as np
import warnings
import pandas as pd
import os 
import torch
from fastai.vision.data import *
import fastai
from pathlib import Path
from shutil import copyfile
import random
import shutil
from PIL import Image
from datetime import datetime
from hyperopt import hp
from hyperopt import fmin, tpe, space_eval
from hyperopt import  Trials
import torch.nn as nn
from datetime import datetime
from time import mktime
import time
from sklearn.model_selection import RepeatedKFold

In [2]:
def get_float_label(x):
    y = str(x)[(len(path_str) + 1):(len(path_str) + 3)]
    dash_search = re.search("/", y)
    if dash_search != None:
        y = y[:dash_search.span()[0]]
    y = float(y)
    return y

In [3]:
path_str = "/media/revilo/oliver-kool-2/pawpularity_data/train_for_resnet"
path = Path(path_str)

warnings.filterwarnings('ignore')

class L1LossFlat(nn.L1Loss):
#“Mean Absolute Error Loss”
  def forward(self, input:Tensor, target:Tensor) -> Rank0Tensor:
    return super().forward(input.view(-1), target.view(-1))


In [4]:
(label,
     size,
    weight_dec, 
    div_factor,
    mom1, 
    mom2, 
    max_rotate,
    max_zoom,
    p_affine,
    max_lighting, 
    max_warp,
    loss_fn,
    model) = ('MSE loss', 
 224, 
 0.3806114553011142, 
 1.6043067786085103, 
 0.7412407050621035,
 0.9707736130395528,
 208.78583621669668,
 2.4716600773278135,
 0.5694889423615632,
 0.2391272797023367,
 0.10510357270731724,
 MSELossFlat,
 models.resnet34
)
 

## Train with the best parameters on 10 epochs and random split

In [5]:
current_dir = ! pwd
current_dir = current_dir[0]

In [6]:
def rmse_basic(x, y):
    return np.sqrt(np.mean((x-y)**2))

In [7]:
no_of_epochs = 10

data = (ImageList.from_folder(path)\
    .random_split_by_pct()\
    .label_from_func(get_float_label, label_cls = 
       FloatList)\
.transform(get_transforms(do_flip=False,
                         max_rotate = max_rotate,
                         max_zoom = max_zoom,
                         p_affine = p_affine,
                         max_lighting = max_lighting,
                         max_warp = max_warp), 
           size=size)\
.databunch())

data.normalize(imagenet_stats) 

learn = cnn_learner(data, model, model_dir = Path(current_dir))
learn.loss = loss_fn
learn.fit_one_cycle(no_of_epochs,
                    div_factor = div_factor,
                    moms = (mom1, mom2),
                    wd = weight_dec)

interp = ClassificationInterpretation.from_learner(learn)
rmse_res = rmse_basic(interp.preds.numpy().reshape(-1), 
    interp.y_true.numpy()) 

print("RMSE: " + str(rmse_res))


RMSE: 17.420443


In [8]:
rmse_res = rmse_basic(interp.preds.numpy().reshape(-1), 
    interp.y_true.numpy()) 

print("RMSE: " + str(rmse_res))

RMSE: 17.420443


In [9]:
# save model
t1 = time.localtime()
dt = datetime.fromtimestamp(mktime(t1))
learn.save(f'resnet_models/{str(dt)}_{np.round(rmse_res, 2)}.pt')

In [45]:
test_dir = "/media/revilo/oliver-kool-2/pawpularity_data/test"
test_path = Path(test_dir)
test = ImageList.from_folder(test_path)
data.add_test(test)

In [22]:
predictions, *_ = learn.get_preds(DatasetType.Test)

In [93]:
# write kaggle submission file
submission_df = pd.DataFrame( index = None,
                             data = [
    [get_name(x) for x in data.test_dl.dataset.items],
    predictions.numpy().flatten()]).T

In [94]:
def get_name(x):
    x = str(x)
    y = x.split("/")
    y = y[len(y) - 1]
    y = y[:(len(y) - 4)]
    return(y)

In [95]:
submission_df.columns = ["Id", "Pawpularity"]

In [96]:
submission_df

Id Pawpularity
0  c978013571258ed6d4637f6e8cc9d6a3     37.8735
1  b03f7041962238a7c9d6537e22f9b017     37.6429
2  e0de453c1bffc20c22b072b34b54e50f     36.8432
3  4e429cead1848a298432a0acad014c9d     37.5755
4  43a2262d7738e3d420d453815151079e     37.9716
5  8f49844c382931444e68dffbe20228f4     37.8716
6  4128bae22183829d2b5fea10effdb0c3     38.7123
7  80bc3ccafcc51b66303c2c263aa38486     37.9077

In [97]:
submission_df.to_csv("submission/submission.csv", index = None)